## This is starter code for single point prediction with CNNs

In [ ]:
num_augmentations = None

In [ ]:
# # uncomment if not running with papermill
# # this does not have to be equal to total number of augmentations
# # used during training
# num_augmentations = 2

In [ ]:
import os
import glob
import joblib

# common math imports
import numpy as np

# common torch imports
import torch
from torch.utils.data import Dataset, DataLoader

# common sklearn imports 
from sklearn.preprocessing import StandardScaler

## Load and process data

In [ ]:
main_path = 'data'
train_folders = [f'{main_path}/train',
                 # f'{main_path}/rain-sounds', f'{main_path}/colored-noise'
]

# ---------- TRAINING DATA ----------

num_augmentations = int(num_augmentations)

X_list = []

for folder in train_folders:
    # find files like X_1000.npy, X_2000.npy, etc.
    X_files = sorted(glob.glob(os.path.join(folder, "X_original_*.npy")))

    for xf in X_files:
        X_list.append(np.load(xf))

for n in range(1, num_augmentations):

    for folder in train_folders:
        # find files like X_1000.npy, X_2000.npy, etc.
        X_files = sorted(glob.glob(os.path.join(folder, f"X_augmented_run{n}_*.npy")))

        for xf in X_files:
            X_list.append(np.load(xf))

# Stack into arrays
X_train = np.vstack(X_list)

# Cleanup
del X_list


In [ ]:
# --- SCALE THE DATA ---
BA, FR, TI = X_train.shape
scaler = StandardScaler()

# Wrap it in a Dataset
class NumpyDataset(Dataset):
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]  # returns NumPy arrays

# training data

X_train_2d = X_train.reshape(BA, FR * TI)
del X_train

dataset = NumpyDataset(X_train_2d)
loader = DataLoader(dataset, batch_size=1000, shuffle=True)
itr = 0
for batch in loader:
    itr += 1
    if itr % 10 == 0:
        print(f'Processed {itr} batches')
    # Convert batch to numpy array for StandardScaler
    scaler.partial_fit(batch)
print()
print('Fit the standard scaler')
print()

# --- SAVE SCALER ---
scaler_path = os.path.join(main_path, "standard_scaler.joblib")
joblib.dump(scaler, scaler_path)
print(f"Saved scaler to {scaler_path}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
Fit the standard scaler
Saved scaler to data/standard_scaler.joblib
